In [1]:
import pandas as pd
import numpy as np

In [4]:
chair = pd.read_excel('chair.xlsx')
chair.tail()

,링크,브랜드명,제품명,가격,별점,리뷰 갯수,헤드레스트 유무,팔걸이 유무,요추지지대 유무,높이 조절 레버 유무,틸팅 여부,등받이 곧/꺾,h8_지면-좌석 높이_MIN,h8_지면-좌석 높이_MAX,b3_좌석 가로 길이,t4_좌석 세로 길이 일반,b4_등받이 가로 길이,h7_등받이 세로 길이
90,https://ohou.se/productions/444813/selling?aff...,다니카,파스텔 접이식 팔걸이 사무용 컴퓨터 메쉬 프리미엄 의자,46900,4.6,780,X,O,X,O,X,꺾,410.0,NaN,NaN,NaN,NaN,495.0
91,https://ohou.se/productions/2431199/selling?af...,포네라,투투체어 사무실 회의실 학원 매장 카페 미드센추리 의자 실버 오피스체어,29900,4.6,68,X,X,X,X,X,꺾,480.0,480.0,480.0,540.0,480.0,350.0
92,https://ohou.se/productions/1452145/selling?af...,체어팩토리,발렌 체어 쿠션 컴퓨터 의자 사무용 책상 사무실 학생 바퀴 편한 패브릭 팔걸이,109000,4.4,109,X,O,X,O,O,곧,360.0,445.0,380.0,475.0,630.0,535.0
93,https://ohou.se/productions/1831442/selling?af...,네오체어,[무료배송]에이스시리즈 일반형(801) 아마존최다판매 사무용메쉬의자,51900,4.7,155,X,O,O,O,O,꺾,410.0,530.0,480.0,500.0,440.0,480.0
94,https://ohou.se/productions/3438753/selling?af...,16피스,KM 홈앤오피스 컴퓨터의자,33900,4.5,8,O,O,O,O,X,꺾,450.0,550.0,520.0,500.0,650.0,500.0


In [5]:
def analyze_column_distribution(df):
    """
    각 컬럼의 분포를 분석하여 범주형/연속형을 구분하고 
    생성 규칙을 반환합니다.
    """
    column_info = {}
    
    for col in df.columns:
        unique_values = df[col].unique()
        unique_count = len(unique_values)
        
        # 범주형 vs 연속형 판단 기준
        # 1. unique 값이 전체 행의 50% 미만이면 범주형으로 간주
        # 2. 또는 unique 값이 10개 이하면 범주형으로 간주
        if unique_count < len(df) * 0.5 or unique_count <= 10:
            # 범주형 컬럼
            column_info[col] = {
                'type': 'categorical',
                'values': unique_values.tolist(),
                'probabilities': df[col].value_counts(normalize=True).to_dict()
            }
        else:
            # 연속형 컬럼
            # NaN/None 값을 제외하고 min/max 계산
            valid_values = df[col].dropna()
            
            if len(valid_values) == 0:
                # 모든 값이 NaN인 경우 스킵
                continue
                
            min_val = valid_values.min()
            max_val = valid_values.max()
            
            # 5단위 체크 (NaN이 아닌 값들만 체크)
            all_values = valid_values.unique()
            is_five_unit = all(val % 5 == 0 for val in all_values if isinstance(val, (int, float)))
            
            column_info[col] = {
                'type': 'continuous',
                'min': min_val,
                'max': max_val,
                'is_five_unit': is_five_unit,
                'dtype': df[col].dtype
            }
    
    return column_info

In [ ]:
analyze_column_distribution(chair)

In [6]:
def generate_random_data(column_info, n_rows=100, dummy_columns=None):
    """
    분석된 컬럼 정보를 바탕으로 랜덤 데이터를 생성합니다.
    
    Parameters:
    -----------
    column_info : dict
        컬럼 분석 정보
    n_rows : int
        생성할 행 수
    dummy_columns : dict
        더미 데이터로 채울 컬럼 정보
        예: {'text': ['링크', '브랜드명'], 'numeric': ['가격', '별점']}
    """
    new_data = {}
    
    # 더미 컬럼 리스트 생성
    dummy_text_cols = dummy_columns.get('text', []) if dummy_columns else []
    dummy_numeric_cols = dummy_columns.get('numeric', []) if dummy_columns else []
    all_dummy_cols = dummy_text_cols + dummy_numeric_cols
    
    for col, info in column_info.items():
        if col in all_dummy_cols:
            # 더미 데이터 생성
            if col in dummy_text_cols:
                # 텍스트 더미 데이터
                new_data[col] = [f'dummy_{i+1}' for i in range(n_rows)]
            else:
                # 숫자 더미 데이터
                new_data[col] = np.arange(1, n_rows + 1)
        
        elif info['type'] == 'categorical':
            # 범주형: 기존 분포를 반영하여 랜덤 선택
            values = list(info['probabilities'].keys())
            probs = list(info['probabilities'].values())
            new_data[col] = np.random.choice(values, size=n_rows, p=probs)
            
        elif info['type'] == 'continuous':
            if info['is_five_unit']:
                # 5단위 연속형
                min_unit = int(info['min'] / 5)
                max_unit = int(info['max'] / 5)
                random_units = np.random.randint(min_unit, max_unit + 1, size=n_rows)
                new_data[col] = random_units * 5
            else:
                # 일반 연속형
                if pd.api.types.is_integer_dtype(info['dtype']):
                    new_data[col] = np.random.randint(info['min'], info['max'] + 1, size=n_rows)
                else:
                    new_data[col] = np.random.uniform(info['min'], info['max'], size=n_rows)
    
    return pd.DataFrame(new_data)

In [8]:
def generate_random_rows(df, n_rows=100, dummy_columns=None):
    """
    기존 데이터프레임의 분포를 분석하고 새로운 랜덤 데이터를 생성합니다.
    
    Parameters:
    -----------
    df : pd.DataFrame
        분석할 원본 데이터프레임
    n_rows : int
        생성할 행 수
    dummy_columns : dict
        더미 데이터로 채울 컬럼 정보
        예: {'text': ['링크', '브랜드명', '제품명'], 
             'numeric': ['가격', '별점', '리뷰갯수']}
    
    Returns:
    --------
    pd.DataFrame
        새로 생성된 랜덤 데이터프레임
    """
    # 1. 각 컬럼의 분포 분석
    print("컬럼 분포 분석 중...")
    column_info = analyze_column_distribution(df)
    
    # 더미 컬럼 정보 출력
    if dummy_columns:
        all_dummy = dummy_columns.get('text', []) + dummy_columns.get('numeric', [])
        print(f"\n더미 데이터로 처리될 컬럼: {all_dummy}")
    
    # 분석 결과 출력
    print("\n=== 컬럼 분석 결과 ===")
    for col, info in column_info.items():
        if dummy_columns and col in (dummy_columns.get('text', []) + dummy_columns.get('numeric', [])):
            print(f"{col}: [더미 데이터로 대체]")
        elif info['type'] == 'categorical':
            print(f"{col}: 범주형, 값: {info['values']}")
        else:
            print(f"{col}: 연속형, 범위: {info['min']} ~ {info['max']}", end="")
            if info['is_five_unit']:
                print(" (5단위)")
            else:
                print()
            # NaN 값이 있었는지 확인
            nan_count = df[col].isna().sum()
            if nan_count > 0:
                print(f"      (원본 데이터에 {nan_count}개의 NaN 값 존재 - 무시됨)")
    
    # 2. 랜덤 데이터 생성
    print(f"\n{n_rows}개의 새로운 행 생성 중...")
    new_df = generate_random_data(column_info, n_rows, dummy_columns)
    
    return new_df

In [ ]:
# 사용 예시
# if __name__ == "__main__":
#     # 예시 데이터프레임 생성 (NaN 값 포함)
#     sample_df = pd.DataFrame({
#         'A': ['O', 'X', 'O', 'O', 'X', 'O', 'X', 'O'],
#         'B': ['곧', '꺾', '곧', '곧', '꺾', '곧', '곧', '꺾'],
#         'C': [200, 350, None, 250, 700, 400, 950, 600],  # NaN 값 포함
#         'D': ['가', '나', '다', '가', '나', '가', '다', '나'],
#         'E': [10.5, None, 15.7, 18.2, None, 19.5, 25.0, 30.2],  # NaN 값 포함
#         'F': [100, 200, None, None, 500, 600, None, 800],  # 5단위 컬럼 with NaN
#         '링크': ['http://a.com', 'http://b.com', 'http://c.com', 'http://d.com', 
#                 'http://e.com', 'http://f.com', 'http://g.com', 'http://h.com'],
#         '브랜드명': ['브랜드A', '브랜드B', '브랜드C', '브랜드A', 
#                    '브랜드B', '브랜드C', '브랜드A', '브랜드B'],
#         '제품명': ['제품1', '제품2', '제품3', '제품4', 
#                  '제품5', '제품6', '제품7', '제품8'],
#         '가격': [10000, 20000, None, 30000, 25000, 18000, 22000, 35000],
#         '별점': [4.5, 3.8, 4.2, None, 3.5, 4.0, 4.7, 4.3],
#         '리뷰갯수': [120, 45, 89, 230, None, 150, 180, 95]
#     })
    
# print("원본 데이터프레임 (NaN 값 포함):")
# print(chair)
# print()

# 더미 컬럼 정의
dummy_columns = {
    'text': ['링크', '브랜드명', '제품명'],
    'numeric': ['가격', '별점', '리뷰갯수']
}

# 새로운 랜덤 데이터 생성
new_df = generate_random_rows(chair, n_rows=20, dummy_columns=dummy_columns)

print("\n생성된 랜덤 데이터프레임:")
print(new_df)
print()

# 생성된 데이터 검증
print("\n=== 생성된 데이터 검증 ===")
for col in new_df.columns:
    print(f"\n{col} 컬럼:")
    if col in ['링크', '브랜드명', '제품명']:  # 텍스트 더미
        print(f"  샘플: {new_df[col].iloc[:3].tolist()}")
    elif col in ['가격', '별점', '리뷰갯수']:  # 숫자 더미
        print(f"  샘플: {new_df[col].iloc[:5].tolist()}")
    elif col in ['A', 'B', 'D']:  # 범주형
        print(f"  고유값: {sorted(new_df[col].unique())}")
        print(f"  분포:\n{new_df[col].value_counts()}")
    else:  # 연속형
        print(f"  범위: {new_df[col].min()} ~ {new_df[col].max()}")
        if col in ['C', 'F']:  # 5단위 체크
            print(f"  5단위 확인: {all(val % 5 == 0 for val in new_df[col])}")

In [11]:
new_df

,링크,브랜드명,제품명,가격,별점,리뷰 갯수,헤드레스트 유무,팔걸이 유무,요추지지대 유무,높이 조절 레버 유무,틸팅 여부,등받이 곧/꺾,h8_지면-좌석 높이_MIN,h8_지면-좌석 높이_MAX,b3_좌석 가로 길이,t4_좌석 세로 길이 일반,b4_등받이 가로 길이,h7_등받이 세로 길이
0,dummy_1,dummy_1,dummy_1,1,1,2840,O,O,X,O,X,곧,420.0,495.0,470.0,485.0,515.0,500.0
1,dummy_2,dummy_2,dummy_2,2,2,5875,X,O,X,O,O,곧,450.0,535.0,490.0,500.0,455.0,480.0
2,dummy_3,dummy_3,dummy_3,3,3,10335,O,O,X,O,O,꺾,280.0,550.0,490.0,480.0,450.0,500.0
3,dummy_4,dummy_4,dummy_4,4,4,7365,O,O,X,O,O,곧,430.0,490.0,480.0,480.0,505.0,520.0
4,dummy_5,dummy_5,dummy_5,5,5,4440,X,O,X,X,X,곧,450.0,505.0,500.0,400.0,480.0,800.0
5,dummy_6,dummy_6,dummy_6,6,6,9800,O,O,X,O,X,꺾,430.0,510.0,450.0,435.0,440.0,470.0
6,dummy_7,dummy_7,dummy_7,7,7,3395,O,O,X,X,O,꺾,430.0,380.0,580.0,480.0,480.0,500.0
7,dummy_8,dummy_8,dummy_8,8,8,7915,X,O,O,O,O,꺾,450.0,490.0,460.0,530.0,450.0,370.0
8,dummy_9,dummy_9,dummy_9,9,9,3835,X,O,X,O,O,꺾,450.0,520.0,500.0,490.0,505.0,610.0
9,dummy_10,dummy_10,dummy_10,10,10,4375,O,O,X,O,O,꺾,480.0,470.0,455.0,480.0,470.0,520.0


In [ ]:
# 더미데이터 100만 개 생성
chair_dummy = generate_random_rows(chair, n_rows=1000000, dummy_columns=dummy_columns)

컬럼 분포 분석 중...

더미 데이터로 처리될 컬럼: ['링크', '브랜드명', '제품명', '가격', '별점', '리뷰갯수']

=== 컬럼 분석 결과 ===
링크: [더미 데이터로 대체]
브랜드명: [더미 데이터로 대체]
제품명: [더미 데이터로 대체]
가격: [더미 데이터로 대체]
별점: [더미 데이터로 대체]
리뷰 갯수: 연속형, 범위: 8 ~ 10692 (5단위)
헤드레스트 유무: 범주형, 값: ['X', 'O']
팔걸이 유무: 범주형, 값: ['O', 'X']
요추지지대 유무: 범주형, 값: ['X', 'O']
높이 조절 레버 유무: 범주형, 값: ['O', 'X']
틸팅 여부: 범주형, 값: ['X', 'O']
등받이 곧/꺾: 범주형, 값: ['꺾', '곧']
h8_지면-좌석 높이_MIN: 범주형, 값: [410.0, 430.0, 450.0, 435.0, 520.0, 455.0, 440.0, 400.0, 425.0, 500.0, 420.0, 470.0, 460.0, nan, 390.0, 405.0, 480.0, 475.0, 280.0, 426.0, 360.0]
h8_지면-좌석 높이_MAX: 범주형, 값: [nan, 520.0, 530.0, 435.0, 455.0, 510.0, 500.0, 495.0, 480.0, 485.0, 490.0, 560.0, 580.0, 540.0, 590.0, 570.0, 505.0, 450.0, 550.0, 555.0, 535.0, 470.0, 460.0, 380.0, 458.0, 488.0, 595.0, 445.0]
b3_좌석 가로 길이: 범주형, 값: [nan, 480.0, 520.0, 460.0, 495.0, 470.0, 430.0, 655.0, 505.0, 500.0, 490.0, 550.0, 420.0, 450.0, 455.0, 510.0, 440.0, 580.0, 630.0, 380.0]
t4_좌석 세로 길이 일반: 범주형, 값: [nan, 500.0, 460.0, 480.0, 485.0, 490.0, 53

In [ ]:
chair_dummy.to_excel('chair_dummy.xlsx', index=False)

In [19]:
# 검증 (사람하고 맞는 의자가 얼마나 있는가)

p1 = pd.read_csv('../person.csv')
p2 = pd.read_excel('../person_test.xlsx')

In [13]:
c1 = pd.read_excel('../chair.xlsx')
c2 = pd.read_excel('../chair_dummy.xlsx')
chair = pd.concat([c1, c2], axis=0)

In [20]:
p1.columns, chair.columns

(Index(['image-name', 'human-height', 'A_Buttock-popliteal length',
        'B_Popliteal-height', 'C_Hip-breadth', 'F_Sitting-height',
        'G_Shoulder-breadth'],
       dtype='object'),
 Index(['링크', '브랜드명', '제품명', '가격', '별점', '리뷰 갯수', '헤드레스트 유무', '팔걸이 유무',
        '요추지지대 유무', '높이 조절 레버 유무', '틸팅 여부', '등받이 곧/꺾', 'h8_지면-좌석 높이_MIN',
        'h8_지면-좌석 높이_MAX', 'b3_좌석 가로 길이', 't4_좌석 세로 길이 일반', 'b4_등받이 가로 길이',
        'h7_등받이 세로 길이'],
       dtype='object'))

In [7]:
import pandas as pd
import numpy as np
from typing import List, Dict, Tuple
import warnings
warnings.filterwarnings('ignore')

class ChairPersonMatcher:
    """의자와 사람의 신체 데이터를 매칭하여 추천하는 시스템"""
    
    def __init__(self):
        self.person_df = None
        self.chair_df = None
        self.matching_results = None
        
        # 매칭 조건 정의
        self.matching_conditions = {
            'seat_depth': {
                'person_col': 'A_Buttock-popliteal length',
                'chair_col': 't4_좌석 세로 길이 일반',
                'condition': 'less_than',  # t4 < A
                'description': '좌석 깊이 (허벅지 길이)',
                'weight': 20
            },
            'seat_height': {
                'person_col': 'B_Popliteal-height',
                'chair_cols': ['h8_지면-좌석 높이_MIN', 'h8_지면-좌석 높이_MAX'],
                'condition': 'approximate',  # h8 ≈ B (범위 내)
                'tolerance': 50,  # ±50mm 허용
                'description': '좌석 높이',
                'weight': 25
            },
            'seat_width': {
                'person_col': 'C_Hip-breadth',
                'chair_col': 'b3_좌석 가로 길이',
                'condition': 'greater_than',  # b3 > C
                'description': '좌석 너비 (엉덩이 폭)',
                'weight': 20
            },
            'backrest_height': {
                'person_col': 'F_Sitting-height',
                'chair_col': 'h7_등받이 세로 길이',
                'condition': 'less_than',  # h7 < F
                'description': '등받이 높이',
                'weight': 20
            },
            'backrest_width': {
                'person_col': 'G_Shoulder-breadth',
                'chair_col': 'b4_등받이 가로 길이',
                'condition': 'greater_equal',  # b4 ≥ G
                'description': '등받이 너비 (어깨 너비)',
                'weight': 15
            }
        }
    
    def load_data(self, person_file: str, chair_file: str, person_test_file: str = None):
        """데이터 로드 및 전처리"""
        # Person 데이터 로드
        if person_file.endswith('.csv'):
            self.person_df = pd.read_csv(person_file)
        else:
            self.person_df = pd.read_excel(person_file)
        
        # Person test 데이터가 있으면 추가
        if person_test_file:
            if person_test_file.endswith('.csv'):
                person_test_df = pd.read_csv(person_test_file)
            else:
                person_test_df = pd.read_excel(person_test_file)
            self.person_df = pd.concat([self.person_df, person_test_df], ignore_index=True)
        
        # Chair 데이터 로드
        if chair_file.endswith('.csv'):
            self.chair_df = pd.read_csv(chair_file)
        else:
            self.chair_df = pd.read_excel(chair_file)
        
        # 단위 변환 (person: cm → mm)
        self._convert_units()
        
        print(f"데이터 로드 완료:")
        print(f"- 사람 수: {len(self.person_df)}명")
        print(f"- 의자 수: {len(self.chair_df)}개")
        
        # 필요한 컬럼 확인
        self._check_required_columns()
    
    def _convert_units(self):
        """사람 데이터의 단위를 cm에서 mm로 변환"""
        convert_cols = ['A_Buttock-popliteal length', 'B_Popliteal-height', 
                       'C_Hip-breadth', 'F_Sitting-height', 'G_Shoulder-breadth']
        
        for col in convert_cols:
            if col in self.person_df.columns:
                self.person_df[col] = self.person_df[col] * 10
    
    def _check_required_columns(self):
        """필요한 컬럼 존재 여부 확인"""
        print("\n컬럼 확인:")
        
        # Person 컬럼 확인
        print("\nPerson 데이터 필수 컬럼:")
        for condition in self.matching_conditions.values():
            col = condition.get('person_col')
            if col and col in self.person_df.columns:
                print(f"✓ {col}")
            else:
                print(f"✗ {col} (없음)")
        
        # Chair 컬럼 확인
        print("\nChair 데이터 필수 컬럼:")
        for condition in self.matching_conditions.values():
            if 'chair_col' in condition:
                col = condition['chair_col']
                if col in self.chair_df.columns:
                    print(f"✓ {col}")
                else:
                    print(f"✗ {col} (없음)")
            elif 'chair_cols' in condition:
                for col in condition['chair_cols']:
                    if col in self.chair_df.columns:
                        print(f"✓ {col}")
                    else:
                        print(f"✗ {col} (없음)")
    
    def _check_condition(self, person_value: float, chair_value: float, 
                        condition_type: str, tolerance: float = None) -> Tuple[bool, float]:
        """조건 체크 및 점수 계산"""
        if pd.isna(person_value) or pd.isna(chair_value):
            return False, 0
        
        if condition_type == 'less_than':
            if chair_value < person_value:
                # 차이가 클수록 좋음 (최대 20mm까지 고려)
                diff = person_value - chair_value
                score = min(diff / 20, 1.0)
                return True, score
            return False, 0
        
        elif condition_type == 'greater_than':
            if chair_value > person_value:
                # 여유가 있을수록 좋음 (최대 50mm까지 고려)
                diff = chair_value - person_value
                score = min(diff / 50, 1.0)
                return True, score
            return False, 0
        
        elif condition_type == 'greater_equal':
            if chair_value >= person_value:
                # 정확히 맞거나 약간 클수록 좋음
                diff = chair_value - person_value
                if diff == 0:
                    score = 1.0
                else:
                    score = max(0, 1 - diff / 30)
                return True, score
            return False, 0
        
        elif condition_type == 'approximate':
            diff = abs(chair_value - person_value)
            if diff <= tolerance:
                # 차이가 적을수록 높은 점수
                score = 1 - (diff / tolerance)
                return True, score
            return False, 0
        
        return False, 0
    
    def match_person_to_chairs(self, person_idx: int) -> List[Dict]:
        """특정 사람에 대해 모든 의자 매칭"""
        person = self.person_df.iloc[person_idx]
        matches = []
        
        for chair_idx, chair in self.chair_df.iterrows():
            match_result = {
                'chair_idx': chair_idx,
                'brand': chair.get('브랜드명', ''),
                'product': chair.get('제품명', ''),
                'price': chair.get('가격', 0),
                'rating': chair.get('별점', 0),
                'reviews': chair.get('리뷰 갯수', 0),
                'matched_conditions': [],
                'unmatched_conditions': [],
                'total_score': 0,
                'match_count': 0
            }
            
            # 각 조건 체크
            for cond_name, condition in self.matching_conditions.items():
                person_val = person[condition['person_col']]
                
                if cond_name == 'seat_height':
                    # 높이 조절 범위 고려
                    h8_min = chair.get(condition['chair_cols'][0])
                    h8_max = chair.get(condition['chair_cols'][1])
                    
                    if pd.notna(h8_min):
                        # MAX 값이 없으면 MIN + 100mm로 가정
                        if pd.isna(h8_max):
                            h8_max = h8_min + 100
                        
                        # 범위 내에 있는지 체크
                        if h8_min <= person_val <= h8_max:
                            # 중간값과의 차이로 점수 계산
                            mid_height = (h8_min + h8_max) / 2
                            is_match, score = self._check_condition(
                                person_val, mid_height, 'approximate', 
                                condition.get('tolerance', 50)
                            )
                            chair_val = f"{h8_min}-{h8_max}"
                        else:
                            # 범위 밖이면 가장 가까운 값과 비교
                            closest = h8_min if person_val < h8_min else h8_max
                            is_match, score = self._check_condition(
                                person_val, closest, 'approximate',
                                condition.get('tolerance', 50)
                            )
                            chair_val = f"{h8_min}-{h8_max}"
                    else:
                        is_match, score = False, 0
                        chair_val = None
                else:
                    chair_val = chair.get(condition['chair_col'])
                    is_match, score = self._check_condition(
                        person_val, chair_val, condition['condition'],
                        condition.get('tolerance')
                    )
                
                if is_match and chair_val is not None:
                    match_result['matched_conditions'].append({
                        'type': condition['description'],
                        'person_value': person_val,
                        'chair_value': chair_val,
                        'score': score
                    })
                    match_result['total_score'] += score * condition['weight']
                    match_result['match_count'] += 1
                elif chair_val is not None:
                    match_result['unmatched_conditions'].append({
                        'type': condition['description'],
                        'person_value': person_val,
                        'chair_value': chair_val
                    })
            
            # 추가 점수: 별점과 리뷰
            if match_result['rating']:
                match_result['total_score'] += match_result['rating'] * 2
            if match_result['reviews'] and match_result['reviews'] > 100:
                match_result['total_score'] += 5
            
            if match_result['match_count'] > 0:
                matches.append(match_result)
        
        # 점수 순으로 정렬
        matches.sort(key=lambda x: (x['match_count'], x['total_score']), reverse=True)
        return matches
    
    def analyze_all_persons(self):
        """모든 사람에 대해 매칭 분석"""
        self.matching_results = {}
        
        for idx in range(len(self.person_df)):
            person_name = self.person_df.iloc[idx]['image-name']
            matches = self.match_person_to_chairs(idx)
            self.matching_results[person_name] = {
                'person_idx': idx,
                'matches': matches
            }
    
    def get_recommendations(self, person_name: str, top_n: int = 5) -> pd.DataFrame:
        """특정 사람에 대한 상위 N개 추천 의자"""
        if person_name not in self.matching_results:
            return pd.DataFrame()
        
        matches = self.matching_results[person_name]['matches'][:top_n]
        
        recommendations = []
        for match in matches:
            rec = {
                '브랜드': match['brand'],
                '제품명': match['product'],
                '가격': f"{match['price']:,}원" if match['price'] else '-',
                '별점': match['rating'],
                '리뷰수': match['reviews'],
                '매칭조건수': f"{match['match_count']}/5",
                '점수': f"{match['total_score']:.1f}",
                '매칭된조건': ', '.join([c['type'] for c in match['matched_conditions']])
            }
            recommendations.append(rec)
        
        return pd.DataFrame(recommendations)
    
    def print_detailed_analysis(self, person_name: str, top_n: int = 3):
        """특정 사람에 대한 상세 분석 출력"""
        if person_name not in self.matching_results:
            print(f"{person_name}의 데이터를 찾을 수 없습니다.")
            return
        
        person_idx = self.matching_results[person_name]['person_idx']
        person = self.person_df.iloc[person_idx]
        matches = self.matching_results[person_name]['matches']
        
        print(f"\n{'='*70}")
        print(f"[{person_name}] 신체 정보 및 의자 추천")
        print(f"{'='*70}")
        
        # 신체 정보 출력
        print("\n📊 신체 측정값:")
        print(f"  - 키: {person['human-height']:.1f}cm")
        print(f"  - 허벅지 길이: {person['A_Buttock-popliteal length']:.0f}mm")
        print(f"  - 앉은 다리 높이: {person['B_Popliteal-height']:.0f}mm")
        print(f"  - 엉덩이 폭: {person['C_Hip-breadth']:.0f}mm")
        print(f"  - 앉은 키: {person['F_Sitting-height']:.0f}mm")
        print(f"  - 어깨 너비: {person['G_Shoulder-breadth']:.0f}mm")
        
        print(f"\n🪑 추천 의자 TOP {top_n}:")
        
        if not matches:
            print("  매칭되는 의자가 없습니다.")
            return
        
        for i, match in enumerate(matches[:top_n], 1):
            print(f"\n  {i}. {match['brand']} - {match['product']}")
            print(f"     💰 가격: {match['price']:,}원")
            print(f"     ⭐ 별점: {match['rating']} (리뷰: {match['reviews']}개)")
            print(f"     📈 매칭 점수: {match['total_score']:.1f}점")
            print(f"     ✅ 매칭 조건: {match['match_count']}/5개")
            
            if match['matched_conditions']:
                print("     충족 조건:")
                for cond in match['matched_conditions']:
                    print(f"       - {cond['type']}: ", end="")
                    if isinstance(cond['chair_value'], str) and '-' in cond['chair_value']:
                        print(f"의자 {cond['chair_value']}mm (사람 {cond['person_value']:.0f}mm)")
                    else:
                        print(f"의자 {cond['chair_value']:.0f}mm vs 사람 {cond['person_value']:.0f}mm")
        
        print(f"\n총 {len(matches)}개의 의자가 매칭되었습니다.")
    
    def get_summary_statistics(self) -> pd.DataFrame:
        """전체 매칭 통계"""
        summary = []
        
        for person_name, result in self.matching_results.items():
            matches = result['matches']
            
            # 조건별 매칭 수 계산
            condition_counts = {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0}
            for match in matches:
                count = match['match_count']
                condition_counts[count] = condition_counts.get(count, 0) + 1
            
            summary.append({
                '이름': person_name,
                '매칭의자수': len(matches),
                '5개조건': condition_counts.get(5, 0),
                '4개조건': condition_counts.get(4, 0),
                '3개조건': condition_counts.get(3, 0),
                '2개조건': condition_counts.get(2, 0),
                '1개조건': condition_counts.get(1, 0),
                '최고점수': matches[0]['total_score'] if matches else 0
            })
        
        return pd.DataFrame(summary)


# 사용 예시
if __name__ == "__main__":
    # 매처 초기화
    matcher = ChairPersonMatcher()
    
    # 데이터 로드
    matcher.load_data(
        person_file='person.csv',
        chair_file='chair_dummy.xlsx',
        person_test_file='person_test.xlsx'
    )
    
    # 모든 사람 분석
    matcher.analyze_all_persons()
    
    # 전체 요약 통계
    summary = matcher.get_summary_statistics()
    print("\n전체 매칭 요약:")
    print(summary)
    
    # 특정 사람 상세 분석 (예: person_test.xlsx의 사람들)
    test_persons = ['할머니', '다온', '민아', '재원']
    for person_name in test_persons:
        matcher.print_detailed_analysis(person_name, top_n=3)
    
    # 특정 사람의 추천 결과를 DataFrame으로 받기
    recommendations = matcher.get_recommendations('재원', top_n=5)
    print("\n재원님의 추천 의자 (DataFrame):")
    print(recommendations)

데이터 로드 완료:
- 사람 수: 18명
- 의자 수: 1000000개

컬럼 확인:

Person 데이터 필수 컬럼:
✓ A_Buttock-popliteal length
✓ B_Popliteal-height
✓ C_Hip-breadth
✓ F_Sitting-height
✓ G_Shoulder-breadth

Chair 데이터 필수 컬럼:
✓ t4_좌석 세로 길이 일반
✓ h8_지면-좌석 높이_MIN
✓ h8_지면-좌석 높이_MAX
✓ b3_좌석 가로 길이
✓ h7_등받이 세로 길이
✓ b4_등받이 가로 길이

전체 매칭 요약:
        이름    매칭의자수    5개조건    4개조건    3개조건    2개조건    1개조건          최고점수
0   test10   999355   32533  239686  464814  229651   32671  2.000053e+06
1   test11   999265    3298  142639  607468  220990   24870  1.999933e+06
2   test12  1000000       0  171807  621343  192124   14726  2.000059e+06
3   test13   999264     890   51451  696645  226219   24059  2.000000e+06
4   test14  1000000    5573   97788  376410  423484   96745  1.999530e+06
5   test15  1000000  238324  470975  246296   42256    2149  2.000081e+06
6   test16   992908       0   45311  312970  517822  116805  2.000054e+06
7   test17  1000000   48301  368801  490973   90775    1150  2.000060e+06
8   test18   999722   19679  222264

In [9]:
# from chair_person_matcher import ChairPersonMatcher
import pandas as pd

# def main():
# 매처 초기화
matcher = ChairPersonMatcher()

# 데이터 로드
print("데이터 로딩 중...")
matcher.load_data(
    person_file='person.csv',
    chair_file='chair.xlsx',
    person_test_file='person_test.xlsx'
)

# 모든 사람 분석
print("\n매칭 분석 중...")
matcher.analyze_all_persons()

# 전체 요약 통계
summary = matcher.get_summary_statistics()
print("\n=== 전체 매칭 요약 ===")
print(summary)

# 요약 통계 저장
summary.to_csv('matching_summary.csv', index=False, encoding='utf-8-sig')
print("\n요약 통계가 'matching_summary.csv'에 저장되었습니다.")

# person_test.xlsx의 사람들 상세 분석
test_persons = ['할머니', '다온', '민아', '재원']

# 각 사람별 추천 결과 저장
all_recommendations = []

for person_name in test_persons:
    print(f"\n{person_name}님 분석 중...")
    
    # 상세 분석 출력
    matcher.print_detailed_analysis(person_name, top_n=5)
    
    # 추천 결과 DataFrame 가져오기
    recommendations = matcher.get_recommendations(person_name, top_n=10)
    recommendations['추천대상'] = person_name
    all_recommendations.append(recommendations)

# 모든 추천 결과를 하나의 Excel 파일로 저장
with pd.ExcelWriter('chair_recommendations_basic_rules.xlsx', engine='openpyxl') as writer:
    # 전체 요약
    summary.to_excel(writer, sheet_name='전체요약', index=False)
    
    # 각 사람별 추천
    for person_name in test_persons:
        recommendations = matcher.get_recommendations(person_name, top_n=10)
        recommendations.to_excel(writer, sheet_name=person_name, index=False)

print("\n추천 결과가 'chair_recommendations_basic_rules.xlsx'에 저장되었습니다.")

# 매칭 조건을 모두 만족하는 의자 찾기
print("\n=== 5개 조건 모두 만족하는 의자 ===")
for person_name in test_persons:
    matches = matcher.matching_results[person_name]['matches']
    perfect_matches = [m for m in matches if m['match_count'] == 5]
    
    if perfect_matches:
        print(f"\n{person_name}:")
        for match in perfect_matches[:3]:
            print(f"  - {match['brand']} {match['product']} ({match['price']:,}원)")
    else:
        print(f"\n{person_name}: 5개 조건을 모두 만족하는 의자가 없습니다.")

# if __name__ == "__main__":
#     main()

데이터 로딩 중...
데이터 로드 완료:
- 사람 수: 18명
- 의자 수: 95개

컬럼 확인:

Person 데이터 필수 컬럼:
✓ A_Buttock-popliteal length
✓ B_Popliteal-height
✓ C_Hip-breadth
✓ F_Sitting-height
✓ G_Shoulder-breadth

Chair 데이터 필수 컬럼:
✓ t4_좌석 세로 길이 일반
✓ h8_지면-좌석 높이_MIN
✓ h8_지면-좌석 높이_MAX
✓ b3_좌석 가로 길이
✓ h7_등받이 세로 길이
✓ b4_등받이 가로 길이

매칭 분석 중...

=== 전체 매칭 요약 ===
        이름  매칭의자수  5개조건  4개조건  3개조건  2개조건  1개조건        최고점수
0   test10     95     0    13    41    31    10   82.227780
1   test11     95     0     7    35    37    16   61.157879
2   test12     94     0     8    40    35    11   76.661520
3   test13     95     0     1    41    39    14   66.688728
4   test14     90     0     8    25    31    26   88.509904
5   test15     95    14    32    26    20     3  100.954398
6   test16     91     0     2    19    49    21   77.288254
7   test17     95     5    22    35    20    13   92.322575
8   test18     95     0    11    37    37    10   73.722655
9   test19     95     2    27    31    27     8   74.935924
10  test20     

In [ ]:
# |사람 측정값 (Person DF)|	의자 측정값 (Chair DF)|	조건 (매칭 기준)|	이미지 상|
# |----------|-------------|------------|---------|
# |'A_Buttock-popliteal length' (좌석 깊이 관련) → 허벅지	|'t4_좌석 세로 길이 일반'|	t4 < A|
# |'B_Popliteal height' (앉은 다리 높이)|	'h8_지면-좌석 높이_MIN/MAX'|	h8 ≈ B|
# |'C_Hip breadth' (엉덩이 폭)|	'b3_좌석 가로 길이'|	b3 > C|
# |'F_Sitting height' (앉은 키)|	'h7_등받이 세로 길이'|	h7 < F|
# |'G_Shoulder breadth' (어깨 너비)|	'b4_등받이 가로 길이'|	b4 ≥ G|